In [ ]:
!pip install torch
!pip install torchio
!pip install SimpleITK

     |████████████████████████████████| 163 kB 7.2 MB/s 
     |████████████████████████████████| 48.4 MB 9.2 kB/s 


In [ ]:
import torchio as tio
from os.path import join
import os
import SimpleITK as sitk
import numpy as np
import shutil
import random

In [ ]:
def data_augment(data_path, num_transform_choice=2):
    sample_names = os.listdir(data_path)
    transforms = [
        # spatial
        tio.RandomAnisotropy(),
        tio.RandomElasticDeformation(num_control_points=10),
        tio.RandomBlur(std=(0, 1.5)),
    ]

    for image_folder in sample_names:
        print('Processing .... ', image_folder)
        file_names = os.listdir(join(data_path, image_folder))
        names = {}
        for i in file_names:
            if i.startswith('ROI'):
                names['ROI'] = join(data_path, image_folder, i)
            else:
                split = i.split('_')
                split2 = split[-1].split('.')
                names[split2[0]] = join(data_path, image_folder, i)
        subject_a = tio.Subject(
            t1=tio.ScalarImage(names['t1']),
            t1ce=tio.ScalarImage(names['t1ce']),
            t2=tio.ScalarImage(names['t2']),
            flair=tio.ScalarImage(names['flair']),
            ROI=tio.ScalarImage(names['ROI']),
            label=tio.LabelMap(names['seg'])
        )
        # transform = tio.RandomElasticDeformation(num_control_points=10)
        transform = tio.Compose(random.sample(transforms, num_transform_choice))
        new_subject = transform(subject_a)
        new_folder = 'augmentation_' + image_folder
        if not os.path.isdir(join(data_path, new_folder)):
            os.mkdir(join(data_path, new_folder))
        else:
            shutil.rmtree(join(data_path, new_folder))
            os.mkdir(join(data_path, new_folder))
            
        for i in new_subject:
            sitk.WriteImage(new_subject.get(i).as_sitk(), join(data_path, new_folder, 'sample_' + names[i if i != 'label' else 'seg'].split('/')[-1]))

In [8]:
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')

    LGG = 'drive/MyDrive/BRATS2017/Brats17TrainingData/LGG/'
    data_augment(LGG, 4)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing ....  Brats17_TCIA_642_1
Processing ....  Brats17_TCIA_650_1
Processing ....  Brats17_TCIA_644_1
Processing ....  Brats17_TCIA_653_1
Processing ....  Brats17_TCIA_654_1
Processing ....  Brats17_TCIA_645_1
Processing ....  Brats17_TCIA_634_1
Processing ....  Brats17_TCIA_639_1
Processing ....  Brats17_TCIA_637_1


KeyboardInterrupt: ignored